In [1]:
import qml_essentials.yaqsi as ys
import qml_essentials.operations as op
import numpy as np

In [2]:
def circuit():
    op.H(wires=0)
    op.CX(wires=[0, 1])

obs = [op.PauliZ(wires=0), op.PauliZ(wires=1)]

In [3]:
script = ys.Script(circuit)
script.execute(type="probs", obs=obs)

Array([0.5, 0. , 0. , 0.5], dtype=float64)

In [4]:
n_qubits = 20

def circuit():
    op.H(wires=0)
    for i in range(n_qubits):
        op.CX(wires=[i, i + 1])

obs = [op.PauliZ(wires=i) for i in range(n_qubits)]
script = ys.Script(circuit)
script.execute(type="probs", obs=obs)

Array([0.5, 0. , 0. , ..., 0. , 0. , 0.5], dtype=float64)

In [9]:
n_qubits = 1

def circuit(phi, theta, omega):
    op.Rot(phi, theta, omega, wires=0)

obs = [op.PauliZ(wires=i) for i in range(n_qubits)]
script = ys.Script(circuit)
script.execute(type="expval", obs=obs, args=(np.pi, 1/2*np.pi, 1/4*np.pi))

Array([2.22044605e-16], dtype=float64)